# House Price Prediction Project
### 1. Giải quyết vấn đề:
Sử dụng các mô hình đã học hoặc nâng cao để áp dụng vào bài toán dự đoán giá nhà
### 2. Chọn lọc các đặc trưng:
Chọn các đặc trưng cần thiết để huấn luyện ML Model.
### 3. Chia datasets
- data: dataset
- x: 'data[features]'
- y: target variable 'SalePrice'

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
data = pd.read_csv('train.csv', index_col='Id')

In [3]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 2. Chọn lọc các đặc trưng

In [4]:
#Tránh noise: Chỉ chọn các biến có khả năng giải thích cho giá trị SalePrice

In [5]:
features = ["LotArea","YearBuilt","1stFlrSF","2ndFlrSF","FullBath","BedroomAbvGr","TotRmsAbvGrd"]

# 3. Chia datasets thành X và Y

In [6]:
x = data[features]
y = data["SalePrice"]

In [7]:
x.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
1,8450,2003,856,854,2,3,8
2,9600,1976,1262,0,2,3,6
3,11250,2001,920,866,2,3,6
4,9550,1915,961,756,1,3,7
5,14260,2000,1145,1053,2,4,9


In [8]:
y.head()

Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64

In [9]:
#tách x và y thành x_train, y_train, x_valid, y_valid để tránh bị overfitting

## 4.Model Training

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV

x_train, x_valid, y_train, y_valid = train_test_split(x,y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [11]:
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.1, random_state=1)

In [12]:
model.fit(x_train, y_train)

Lasso(alpha=0.1, random_state=1)

In [13]:
y_pred = model.predict(x_valid)

In [14]:
model.score(x_valid, y_valid)

0.5720203363758133

In [15]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)

LinearRegression()

In [16]:
y_preds = model.predict(x_valid)

In [17]:
linear_model.score(x_valid,y_valid)

0.5720200748089042

In [18]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=1)

In [19]:
dt_model.fit(x_train, y_train)

DecisionTreeRegressor(random_state=1)

In [20]:
y_preds = dt_model.predict(x_valid)

In [21]:
pd.DataFrame({'y': y_valid,'y_preds': y_preds})

,y,y_preds
Id,,
530,200624,335000.0
492,133000,140200.0
460,110000,119000.0
280,192000,207500.0
656,88000,112000.0
...,...,...
327,324000,260400.0
441,555000,451950.0
1388,136000,107500.0


In [22]:
dt_model.score(x_valid, y_valid)

0.5325376245236801

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(x_train, y_train)

RandomForestRegressor(random_state=1)

In [24]:
rf_val_preds = rf_model.predict(x_valid)

In [25]:
rf_val_preds[:5]

array([271690., 155039., 122024., 188915.,  91147.])

In [30]:
rf_model.score(x_valid, y_valid)

0.7842596223072973

In [31]:
gradient_model = GradientBoostingRegressor(random_state=0)
gradient_model.fit(x_train, y_train)

GradientBoostingRegressor(random_state=0)

In [32]:
y_preds =model.predict(x_valid)

In [33]:
gradient_model.score(x_valid,y_valid)

0.8346757461038037

In [34]:
# Khởi tạo và huấn luyện mô hình XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(x_train, y_train)
#reg:squarederror là lựa chọn phổ biến cho bài toán hồi quy với dữ liệu liên tục.
#n_estimators=100 (Số lượng cây quyết định)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
y_pred = model.predict(x_valid)

In [36]:
model.score(x_valid,y_valid)

0.8051888346672058

In [37]:
# Sử dụng pp Cross-Validation để cải thiện mô hình
# Đặt tham số để tìm kiếm

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

In [38]:
# Sử dụng GridSearchCV để tìm các tham số tốt nhất
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.1, ma...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=3, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_squared_error')

In [39]:
# Hiển thị các tham số tốt nhất
print("Best Parameters found: ", grid_search.best_params_)

Best Parameters found:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50}


In [40]:
y_pred = grid_search.best_estimator_.predict(x_valid)

In [41]:
grid_search.best_estimator_.score(x_valid,y_valid)

0.8263283967971802